In [1]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
from IPython.core.display import HTML 
morph = MorphAnalyzer()
import numpy as np

# Подготовка данных

Возьмем 10 тыс статьи с Википедии. 

### Дополнительные стоп-слова:

In [58]:
additional = ["c","а","алло","без","белый","возле","благодаря","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы",\
 "бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше",\
 "ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне",\
 "вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вперёд","вот",\
 "впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё",\
 "второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город",\
 "да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый",\
 "двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю",\
 "дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг",\
 "другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё",\
 "ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем",\
 "здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя",\
 "иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого",\
 "ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто",\
 "куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее",\
 "меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные",\
 "многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь",\
 "моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш",\
 "наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко",\
 "несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто",\
 "но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако",\
 "одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда",\
 "отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока",\
 "пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему",\
 "почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа",\
 "работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами",\
 "самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой",\
 "свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал",\
 "сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем",\
 "спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая",\
 "также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой",\
 "тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут",\
 "ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час",\
 "часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы",\
 "чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот",\
 "эту","я","являюсь"]

In [59]:
stops = set(stopwords.words('russian') + additional)

# чтобы быстрее нормализовать тексты, создадим словарь всех словоформ
# нормазуем каждую 1 раз и положим в словарь
# затем пройдем по текстам и сопоставим каждой словоформе её нормальную форму

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    """
    выбрасываем цифры (не нужны)
    """
    words = re.findall(r"[а-яА-ЯёЁ]+-*[а-яА-ЯёЁ]*", text)
    return words

In [60]:
texts = open('wiki_data.txt', encoding='utf-8').read().splitlines()[:10000]

In [61]:
texts = opt_normalize([tokenize(text.lower()) for text in texts], 16000)

### Изменён порог для получения интерпретируемых биграмм

In [62]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.6)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

# LDA без TF-IDF

In [116]:
dictinary = gensim.corpora.Dictionary(ngrammed_texts)

In [117]:
dictinary.filter_extremes(no_above=0.075, no_below=20)
dictinary.compactify()

In [118]:
print(dictinary)

Dictionary(4170 unique tokens: ['активно', 'берег_река', 'близость', 'возраст', 'данные']...)


In [119]:
corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

In [120]:
lda_1 = gensim.models.LdaMulticore(corpus, 200, alpha=0.11, eta=0.66, id2word=dictinary, eval_every=0, passes=15)
lda_2 = gensim.models.LdaMulticore(corpus, 150, alpha=0.33, eta=0.33, id2word=dictinary, eval_every=0, passes=20)
lda_3 = gensim.models.LdaMulticore(corpus, 100, alpha=0.66, eta='auto', id2word=dictinary, eval_every=0, passes=25)

In [121]:
for index, model in enumerate([lda_1, lda_2, lda_3]):
    print(f"Model #{index+1} stats:")
    print(f"Perplexity: {model.log_perplexity(corpus[:1000])}\n\n")

Model #1 stats:
Perplexity: -10.10597786498808


Model #2 stats:
Perplexity: -10.272879521020124


Model #3 stats:
Perplexity: -10.129609938905116




### Наилучшие результаты показывает модель №1, выводим три лучшие темы.

In [ ]:
"""
ТЕМА 1
(194,
  '0.009*"миссия" + 0.006*"индеец" + 0.002*"поселение" 
  + 0.002*"северный_америка" + 0.002*"племя" 
  + 0.002*"орден" + 0.002*"французский" 
  + 0.002*"христианство" + 0.001*"деятельность" + 0.001*"квебек"')

ТЕМА 2
(193,
  '0.015*"мозг" + 0.011*"поражение" + 0.006*"кровь" 
  + 0.006*"клетка" + 0.006*"ядро" + 0.005*"реакция" 
  + 0.005*"волокно" + 0.005*"больной" + 0.004*"верхний" 
  + 0.004*"возникать"')

ТЕМА 3
(44,
  '0.011*"альбом" + 0.009*"нил" + 0.008*"выпустить" 
  + 0.007*"гитарист" + 0.005*"играть" + 0.004*"американский" 
  + 0.004*"покинуть" + 0.003*"гитара" + 0.003*"участник" 
  + 0.003*"дебютный_альбом"')

"""

In [ ]:
lda_1.print_topics()[:50]

In [132]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda_1, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [133]:
topics = []
for topic_id, topic in lda_1.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [134]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [135]:
coherence_model_lda.get_coherence()

0.47054040356373505

# Добавление tf-idf

In [89]:
from gensim.models import TfidfModel
from gensim.models import ldamodel

In [136]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)
corpus = tfidf[corpus]

In [148]:
lda_4 = gensim.models.LdaModel(corpus, 100, alpha='auto', eta='auto', id2word=dictinary, passes=20)

In [138]:
lda_5 = gensim.models.LdaMulticore(corpus, 125, alpha='symmetric', eta='auto', id2word=dictinary, passes=25)

In [139]:
lda_6 = gensim.models.LdaMulticore(corpus, 150, workers=2, alpha='asymmetric', eta='auto', id2word=dictinary, passes=30)

In [149]:
for index, model in enumerate([lda_4, lda_5, lda_6]):
    print(f"Model #{index+4} stats:")
    print(f"Perplexity: {model.log_perplexity(corpus[:1000])}\n\n")

Model #4 stats:
Perplexity: -20.69278501358015


Model #5 stats:
Perplexity: -49.46851254502912


Model #6 stats:
Perplexity: -38.29892329695855




### Из моделей 4-6 лучший результат показывает модель №4 за счёт автоматического подбора параметров альфа и ета. В целом для tf-idf характерно уменьшение perplexity на данном корпусе.

In [141]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda_4, 
                                                  texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [142]:
topics = []
for topic_id, topic in lda_4.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [143]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [144]:
coherence_model_lda.get_coherence()

0.4117951170713938

### Когерентность топиков снизилась. Это отразилось и на их составе: внутри топиков попадаются слова-"выбросы", в том числе в наиболее когерентных. 

### Состав топиков почти не повторяет предыдущую модель, кроме "спортивной", "военной", "музыкальной" и "административной" (статьи про населённые пункты) тем.

In [ ]:
lda_4.print_topics()

In [ ]:
"""
ТЕМА 1
(13,
  '0.059*"музей" + 0.053*"художник" + 0.031*"выставка" 
  + 0.028*"документ" + 0.023*"картина" + 0.023*"париж" 
  + 0.021*"вариант" + 0.019*"сочинение" + 0.017*"лидер" 
  + 0.016*"журналист"')

ТЕМА 2
(17,
  '0.018*"население" + 0.014*"деревня" 
  + 0.013*"административный_центр" + 0.013*"мужчина" 
  + 0.012*"норвегия" + 0.011*"н" + 0.011*"относиться" 
  + 0.011*"образовать" + 0.010*"площадь" + 0.009*"местный"')

ТЕМА 3
(90,
  '0.038*"медаль" + 0.025*"ссср" + 0.022*"великий" 
  + 0.019*"партия" + 0.018*"международный" + 0.016*"совет" 
  + 0.015*"золотой" + 0.013*"президент" + 0.013*"соревнование" 
  + 0.012*"флаг"')
"""

# NMF

In [7]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [8]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [9]:
vectorizer = CountVectorizer(max_features=1750, min_df=15, max_df=0.15, ngram_range=(1,2))
X = vectorizer.fit_transform(stexts)

Разложим её.

In [14]:
# n_components - главный параметр в NMF, это количество тем. 
# Если данных много, то увеличения этого параметра сильно увеличивает время обучения
model_1 = NMF(l1_ratio=0, solver='mu', alpha=0.2, beta_loss='kullback-leibler', max_iter=250, n_components=150)

In [11]:
model_2 = NMF(l1_ratio=0.75, solver='cd', shuffle=True, beta_loss='frobenius', max_iter=200, n_components=200)

In [16]:
model_3 = NMF(l1_ratio=0.33, init='nndsvd', solver='mu', beta_loss='frobenius', max_iter=150, n_components=250)

In [ ]:
for model in [model_1, model_2, model_3]:
    model.fit(X)

In [22]:
for index, model in enumerate([model_1, model_2, model_3]):
    print(f'Model number {index+1}:')
    print(f'Reconstruction error {model.reconstruction_err_}\n\n')

Model number 1:
Reconstruction error 1516.0672522532923


Model number 2:
Reconstruction error 1006.467619954277


Model number 3:
Reconstruction error 1033.1316375141125




### Лучшую статистику показывает модель №2. Ниже выведены 3 удачные темы.

### Прежние темы (перечислены в tf-idf) остаются, к ним прибавляется несколько новых содержательных тем: "морская", "экспедиционная" (см. ниже).

In [18]:
feat_names = vectorizer.get_feature_names()

Теперь посмотрим на матрицу темы-слова, отсортируем её по строкам и возьмем топ N слов, сопоставив индексы со словарём

In [ ]:
top_words = model_2.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

In [ ]:
"""
Тема 1:
экспедиция  лёд  северный  полюс

Тема 2:
флотилия  капитан  лейтенант  порт

Тема 3:
группа  участник  сингл  выйти

"""

In [23]:
del model_1, model_2, model_3, stexts